## 1.2. Merge DEM tiles to one DEM file

This function is to merge multiple DEM files (*.tif) into one file (.tif).

In [ ]:
import os
import glob
import rasterio
from rasterio.merge import merge

### --- Configuration (User MUST update these paths) --- ###
DEM_SOURCE_DIR = "/media/remap/NO_HEAT_RB/City_Atlanta/Raw/DEM/" # all tif here should be DEM
DEM_OUTPUT_PATH = "/media/remap/NO_HEAT_RB/City_Atlanta/Processed/DEM/output/DEM_merged.tif"
### ---------------------------------------------------- ###

In [ ]:
def merge_all_tiles(dem_source_dir, dem_out_path):
    os.makedirs(os.path.dirname(dem_out_path), exist_ok=True)
    tif_files = []
    for root, dirs, files in os.walk(dem_source_dir):
        for file in files:
            if file.lower().endswith('.tif'):
                tif_files.append(os.path.join(root, file))
    print(f"Found {len(tif_files)} TIFF files.")

    # Open all tiff files and add them to a list
    src_files_to_merge = []
    for fp in tif_files:
        src = rasterio.open(fp)
        src_files_to_merge.append(src)

    # Merge the rasters
    mosaic, out_trans = merge(src_files_to_merge)

    # Update metadata from one of the source files (first one in this case)
    out_meta = src_files_to_merge[0].meta.copy()
    out_meta.update({
        "driver": "GTiff",
        "height": mosaic.shape[1],
        "width": mosaic.shape[2],
        "transform": out_trans
    })

    # Write the mosaic to disk
    with rasterio.open(dem_out_path, "w", **out_meta) as dest:
        dest.write(mosaic)
        print(f"crs is {dest.crs}")
    print(f"Merged raster saved as {dem_out_path}")


    # Close all the open datasets
    for src in src_files_to_merge:
        src.close()

In [ ]:
if __name__ == '__main__':
    merge_all_tiles(DEM_SOURCE_DIR, DEM_OUTPUT_PATH)